## Precarga de librerias y funciones

In [1]:
%run "../recurrentes.ipynb"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.9/543.9 kB 20.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Matplotlib is building the font cache; this may take a moment.


/home/n_alba_dscience/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/n_alba_dscience/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
%run "../funciones.ipynb"

In [3]:
#configuración de la base de datos

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
# Preseteo optuna especificar bucket y nombre del estudio


nombre_archivo = 'optimization_tree.db'
bucket = 'b2/'


estudio_optuna = base_path + 'buckets/' + bucket + 'optimization_tree.db'


# cargar estudio
# a) competencia_02
# b) competencia_02_lags
# c) competencia_02_lags_y_deltas

nombre_estudio = 'competencia_02_lags_deltas_psi_fe_sorted'


In [33]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria
# b) dataset_lags_clase_ternaria
# c) dataset_lags_deltas_y_clase_ternaria

df_train = pd.read_parquet(dataset_lags_deltas_y_clase_ternaria)


# Light GBM

## Formateo pre modelo

In [34]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [35]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202104
mes_test = 202108


In [36]:
dataset = df_train.copy

In [37]:
data = df_train

In [38]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [39]:
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [40]:
# Especificar mes de train y test

df_train = data[data['foto_mes']<=202106]
df_test = data[data['foto_mes']==202108]


In [41]:
df_train = df_train.sort_values(by='foto_mes', ascending=True)

In [42]:
df_test.shape

(165442, 602)

In [45]:
# df_train = df_train.drop(columns=dicc_psi['dataset_lags_deltas_y_clase_ternaria'])
df_test = df_test.drop(columns=dicc_psi['dataset_lags_deltas_y_clase_ternaria'])
df_train = df_train.drop(columns=dicc_psi['lista_light_gbm_feature_importance'])
df_test = df_test.drop(columns=dicc_psi['lista_light_gbm_feature_importance'])


In [46]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [47]:
df_train.head(100)

,numero_de_cliente,foto_mes,active_quarter,cliente_edad,cliente_antiguedad,mactivos_margen,mpasivos_margen,cproductos,tcuentas,mcuenta_corriente_adicional,...,Visa_mpagospesos_delta_2,Visa_mpagosdolares_delta_1,Visa_mpagosdolares_delta_2,Visa_cconsumos_delta_1,Visa_cconsumos_delta_2,Visa_cadelantosefectivo_delta_1,Visa_mpagominimo_delta_1,Visa_mpagominimo_delta_2,clase_peso,clase_binaria
1325131,574728647,201901,1,56,183,563.53,1707.67,9,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0
1842864,880275325,201901,1,32,33,607.79,35.18,7,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0
2230554,772457039,201901,1,37,76,-37.30,551.12,6,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0
1673847,939747494,201901,1,29,92,1356.07,981.85,8,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0
3232104,972259544,201901,1,27,55,-835.93,1148.13,7,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541015,752746674,201901,1,39,137,0.00,7073.43,6,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00002,1
2930919,641262417,201901,1,38,80,-261.74,2445.25,7,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0
541698,765829137,201901,1,43,21,-10.74,815.46,7,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0
2230647,774563099,201901,1,71,32,-85.52,95.09,6,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0


In [48]:
w_train = df_train.loc[X_train.index, 'clase_peso']


### Optimización

In [ ]:
# 4 - Optimizacion

import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
import optuna

# Definir el número de divisiones para TimeSeriesSplit
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)
folds = list(tscv.split(X_train))

def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 200, 10000)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 8000)
    feature_fraction = trial.suggest_float('feature_fraction', 0.3, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.3, 1.0)

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }

    # Asegurarse de que los datos estén ordenados cronológicamente
    X_train_sorted = X_train.sort_index()
    Y_train_sorted = Y_train.loc[X_train_sorted.index]
    w_train_sorted = w_train.loc[X_train_sorted.index]

    train_data = lgb.Dataset(
        X_train_sorted,
        label=Y_train_sorted,
        weight=w_train_sorted
    )

    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=10000,
        feval=lgb_gan_eval,
        folds=folds,          # Pasar los folds generados
        stratified=False,     # Establecer en False
        seed=semillas[0]
    )

    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos la mejor iteración
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5

# Configurar el almacenamiento y el estudio de Optuna
storage_name = rf"sqlite:///{estudio_optuna}"
study_name = nombre_estudio

# Crear el estudio
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

# Definir el callback para early stopping
class EarlyStoppingByImprovement:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_value = None
        self.no_improvement_trials = 0

    def __call__(self, study, trial):
        current_best_value = study.best_trial.value

        if self.best_value is None or current_best_value > self.best_value:
            self.best_value = current_best_value
            self.no_improvement_trials = 0
        else:
            self.no_improvement_trials += 1

        if self.no_improvement_trials >= self.patience:
            print(f"Early stopping: No hay mejora en {self.patience} pruebas consecutivas.")
            study.stop()

# Crear el callback con paciencia de 100
early_stopping_callback = EarlyStoppingByImprovement(patience=100)

# Configurar el nivel de registro de Optuna (opcional)
optuna.logging.set_verbosity(optuna.logging.INFO)

# Optimizar el estudio con el callback personalizado
study.optimize(
    objective,
    n_trials=30000,
)


[I 2024-11-20 05:44:33,694] Using an existing study with name 'competencia_02_lags_deltas_psi_fe_sorted' instead of creating a new one.


## Voting GD

In [ ]:
# 6 - Voting promedio de modelos para reducir varianza

best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")

train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)

df_voting = pd.DataFrame()

for semilla in numeros_random:
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_params['num_leaves'],
    'learning_rate': study.best_params['learning_rate'],
    'min_data_in_leaf': study.best_params['min_data_in_leaf'],
    'feature_fraction': study.best_params['feature_fraction'],
    'bagging_fraction': study.best_params['bagging_fraction'],
    'seed': semilla,
    'verbose': 0
}
    
    
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    df_voting[f'semilla_{semilla}'] = model.predict(X_test)

df_voting['promedio'] = df_voting.mean(axis=1)
    

Mejor cantidad de árboles para el mejor model 300


C:\Users\Admin\AppData\Local\Temp\ipykernel_2908\1798680220.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_voting['promedio'] = df_voting.mean(axis=1)


## Predicción

In [ ]:
# 7 - Predicción del modelo 

y_pred_lgm = df_voting['promedio']
y_pred_labels = (y_pred_lgm > 0.025).astype(int)

In [ ]:
# 8 - Dataframe con las variables más importantes
importancias = model.feature_importance()
nombres = model.feature_name()
df_resultado = pd.DataFrame({
    'numero_de_cliente': X_test['numero_de_cliente'],
    'Predicted': y_pred_labels
})

## Entrega Kaggle

In [ ]:
#Ingresar el path sobre el cual se quiere ingresar el archivo a entregar
entrega = 'entrega_kaggle'
formato = '.csv'
numero = entrega+'_001'+formato
entrega_final = df_resultado.to_csv(rf"{save_path}{numero}}", index=False)
best_params = study.best_params

In [ ]:
cantidad_columnas = df_gbm.shape[1]
message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}"

<!-- ### Kaggle -->

In [ ]:

competition= 'DMEyF-2024-Segunda'

In [ ]:
kaggle.api.competition_submit(competition=competition, file_name= numero, message=message, quiet=False)

### Puntos de corte

In [ ]:

# predicciones = y_pred_lgm

# X_test['Probabilidad'] = predicciones

# tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

# cortes = range(9000,14000,100)

# num_subida_kaggle = 65
# for envios in cortes:
    
#     tb_entrega['Predicted'] = 0
#     tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
#     resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
#     print("Cantidad de clientes {}".format(envios))
    
#     nombre_archivo = 'entrega_0{}.csv'.format(num_subida_kaggle)
#     entrega_final = os.path.join(path, nombre_archivo)
#     resultados.to_csv(entrega_final, index=False)
    
    
#     cantidad_columnas = df_train.shape[1]
#     message = f"{entrega}, cantidad de columnas en el train: {cantidad_columnas}, modelo: LGBM, mejores parametros: {best_params}, mejor iteracion: {best_iter}, archivo: {entrega_final}, punto de corte: {envios}, optimizado con optuna: {study_name}"
    
#     num_subida_kaggle += 1
    
    
    
#     entrega_final = os.path.join(path, nombre_archivo)   
#     competencia = 'dm-ey-f-2024-primera'
#     try:
#         api.competition_submit(file_name=entrega_final,message=message,competition=competencia)
#     except:
#         print(f"Numero máximo de envios, último envio ={num_subida_kaggle}")

In [ ]:
# df_train.shape

In [ ]:
# nombre_modelo = 'lgbm_e_en_abril_p_en_junio_451_features.txt'
# model.save_model(rf"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\modelos_lgbm\{nombre_modelo}")